# Lab Task 3: DDI Baseline
### Authors: David Curto & David Hilazo

The aim of this lab is to implement a handcrafted RULE-based classifier which is able to detect interactions between pairs of drugs and their type. This task is inspired in the Semeval-2013 Task 9 DDI-Extraction.The goal of this task is to obtain a **F1 Score of 0.15 on the Devel set** with just using rules(no ML method).

## Brief description of used linguistic information

In order to detect the interaction between drugs we have used as many linguistic information as we could. At the beginning we only focus on **word level(tokens)** but it obviously was not good enough, so we started using more information like **Part of Speech(PoS)**. With PoS tags we were able to detect how each token was used in that sentence(verb, noun, adjective...) and retrieve more insights about the interaction and discard some forms that may never appear within an interaction. Finally, we include all the information that the Stanford CoreNLP Dependency parser gives, which is all the information mentioned previously but many more. With the parser information we have been able to detect the **relations** and **dependencies** between tokens, which nodes are shared, how they interact and many more features that have been extremely useful for tackling this drug interaction extraction problem.


## Brief description of used/discarded rules

In order to detect the interaction between drugs, we started with a really simple approach that was to look at **common words that appear between the drugs** for each interaction type.

We have included all this rules in the function ``rules_without_dependency``. To search which where the common words for each type we just grouped by interaction type and count the frequency of each word between the pair of drugs in the training dataset, but also considering their frequency in the no interaction case. With this approach we extracted which are the most commond words for each type and built a specific rule for every type.

The next rule we tried was based on the **distance(in terms of words) between the pair of drugs**. The reason behind this rule is that we supposed that drugs which are far away are less likely to interact. After visualising the results for true and false ddis we observed that our supposition was true, however the distance for detecting if true drugs interact or not was too high that is was not really useful at all because there were really few instances classified from this rule. For this reason we decided it to discard it.

At this point, with only the first rules, we were really close to achieve the F1 score of 0.15 on Devel dataset. To improve the performance we started exploring rules using more complex information from the dependency tree. 

Instead of looking at common words between the words, we decided to go further and extract which were the **most common root lemmas** for each interaction type. This rules are based on extracting the lemma of the root from the dependency tree and check if is one of the most common root lemas for each type such as advise, recommend, suggest for *advise* type or enhance, inhibit, block, produce for *effect* type. The reason behind this rule is that sometimes the interaction verb is not between this words but before or after them.

Continuing with the dependency tree we extracted the **common ancestor for the pair of drugs** and analyze it for each type. After analysing the results we came up with one rule for the *int* type that was to look if the common ancestor was "interact" or "interaction" and its postag was a noun or verb type (NN,NNS,VB,VBP).

At this point, we already had achieved the minimum score but, after analyzing the false positives DDIs we added one last rule, that was to **compare if the drug names were the same**. In that case , there couldn't be an interaction. This was a quite simple rule but avoided several misclassification.

Other discarded rules that we tried was analyzing the **heads of each drug**, but this was quite problematic for the compounnd drug names or those drugs that contain text between their names and there were not useful patterns. Another rule that we tried was to focus only on the **common ancestor verbs**, but this was not good enough because it failed to detect nouns formed from verbs such as interaction.

Since we already achieved the minimum score in this lab, we decided not to add more rules even though we have explored some others which will be used in the ML model such as looking at the dependencies of the shortest path.

As an overview of the rules tested and which ones are used and which ones are discarded the following table is used for this purpose

|           Rules  Used       |     Rules     Discarded     |
|:---------------------------:|:---------------------------:|
|  Common words between drugs |    Distance between drugs   |
|      Common root lemmas     |      Head of each drug      |
| Common ancestor (PoS,Lemma) | Common ancestor (verb only) |
|          Drug names         |                             |









## Code with its corresponding comments

**Libraries and modules import**

To implement this DDI detector, we have used just some useful libraries/modules which are the ElementTree XML for parsing the input files which are in XML format and the Stanford CoreNLP Dependency parser that is used for tokenizing the sentences, PoS tagging and parsing them.

**Offset addition**

This function receives the parse of the Stanford CoreNLP Dependency Parser and the text parsed and iterates through the nodes in order to add two more attributes which are:the start and the end position of the token within the sentence.

In [60]:
def add_offset_to_tree(parse, text, offset=0):
    for key in range(len(parse.nodes)):
        value = parse.nodes[key]
        # Check that word exists and its not punctuation
        if value['word'] and value['rel'] != 'punct':
            start = text.find(value['word'], offset)
            parse.nodes[key]['start_off'] = start
            if len(value['word']) > 1:
                parse.nodes[key]['end_off'] = len(value['word']) - 1 + start
            else:
                parse.nodes[key]['end_off'] = len(value['word']) + start
            offset = start + len(value['word'])

        elif value['rel'] == 'punct':
            parse.nodes[key]['start_off'] = offset
            parse.nodes[key]['end_off'] = offset + 1
            offset += 1

    return parse

**Analyze**

This function establishes the connection with the CoreNLPDependencyParser server which is running in the localhost and sends to the server the sentence that wants to parse. This function will return the sentence parsed, tokenized and PoS-tagged with the start and end offset included.


In [61]:
def analyze(stext):
    parser = CoreNLPDependencyParser(url="http://localhost:9000")

    if '\r\n' in stext:
        stext = stext.replace('\r\n', '  ')
    iterator = parser.raw_parse(stext)
    parse = next(iterator)
    #Add the offsets to the tree
    parse = add_offset_to_tree(parse, stext)

    return parse

### Dependengy graph features extraction

**Find common ancestor**

This function finds the common ancestor of the two entities analyzed(pairs of drugs) and if it finds a common parent node between this drug it returns the lemma of the common parent and its postag. 

In [63]:
def find_common_ancestor(analysis, first_index, second_index):
    visited_first = [first_index]
    visited_second = [second_index]

    while not (analysis.root['address'] in visited_first and analysis.root['address'] in visited_second):
        head = analysis.nodes[first_index]['head']
        if head is not None:
            visited_first.append(head)
            first_index = head
        head = analysis.nodes[second_index]['head']
        if head is not None:
            visited_second.append(head)
            second_index = head
        #Find the intersection between the drugs
        intersection = list(set(visited_first) & set(visited_second))
        if intersection:
                return analysis.nodes[intersection[0]]["lemma"], analysis.nodes[intersection[0]]["tag"]

    return 'None', 'None'

**Extract head drugs info**

This function returns the lemma and PoS tag of the head node of each drug. The head node will be set to the first head node which is not in any of the nodes of the Drug 1 and Drug 2. This restriction has been set due to composite drug names that were splitted into different nodes

In [64]:
def extract_head_drugs_info(analysis, nodes_drug1, nodes_drug2):
    drug1_node = nodes_drug1[0]
    drug2_node = nodes_drug2[0]
    counter_d1 = counter_d2 = 0
    head1_found = head2_found = False
    
    while not head1_found or not head2_found:
        # Get the Lemma and Tag of drug's head
        head_drug1 = analysis.nodes[drug1_node["head"]]
        head_drug2 = analysis.nodes[drug2_node["head"]]
        lemma_pos_d1 = (head_drug1["lemma"], head_drug1["tag"])
        lemma_pos_d2 = (head_drug2["lemma"], head_drug2["tag"])
        # Check that the head is not part of the drug(composite names)
        if head_drug1 in nodes_drug1:
            counter_d1 = counter_d1 + 1
            drug1_node = nodes_drug1[counter_d1]
        else:
            head1_found = True
        if head_drug2 in nodes_drug2:
            counter_d2 = counter_d2 + 1
            drug2_node = nodes_drug2[counter_d2]
        else:
            head2_found = True

    return lemma_pos_d1, lemma_pos_d2

**Extract in between text**: This function extracts the text that it is between the drug1 and drug 2(without including the drugs' names in the extracted sentence)

In [1]:
def extract_inbetween_text(analysis, entities, id_e1, id_e2):
    index_drug1 = index_drug2 = 0
    sentece_analysis = [None] * (len(analysis.nodes) - 1)
    for i_node in range(1, len(analysis.nodes)):
        current_node = analysis.nodes[i_node]
        address = current_node["address"]
        word = current_node["word"]
        start = current_node["start_off"]
        end = current_node["end_off"]
        end_drug1 = entities[id_e1][0][1] if len(entities[id_e1] == 1) else entities[id_e1][len(entities[id_e1]) - 1][1]
        if end == end_drug1:
            index_drug1 = i_node - 1
        if start == entities[id_e2][0][0]:
            index_drug2 = i_node - 1
        sentece_analysis[address - 1] = word
    inbetween_text = ' '.join(sentece_analysis[index_drug1:index_drug2])
    return inbetween_text

**Extract distance between drugs** 

This function returns the distance that exists between the end of the first drug and the start of the second drug. The distance is computed in terms of tokens(words).

In [66]:
def extract_distance_between_drugs(nodes_drug1, nodes_drug2):
    max_d1 = max([i["address"] for i in nodes_drug1])
    min_d1 = min([i["address"] for i in nodes_drug2])
    distance = abs(min_d1 - max_d1)
    return distance

**Extract drug names:** 
This function extracts the names of each drug and, in case that it is splitted in multiple nodes, those are joined to form the drug name.

In [67]:
def extract_drug_name(entities,id, nodes_drug):
    drug = []
    for node in nodes_drug:
        if len(entities[id]) < 2:
            drug.append(node["word"])
        else:
            if not (node["start_off"] > entities[id][0][1] and node["end_off"] < entities[id][1][0]):
                drug.append(node["word"])
    return " ".join(drug)

def extract_drug_names(entities, id_e1, id_e2, nodes_drug1, nodes_drug2):
    drug1 = extract_drug_name(entities, id_e1, nodes_drug1)
    drug2 = extract_drug_name(entities, id_e2, nodes_drug2)
    return drug1, drug2

**Extract drug nodes:** This function returns the list of the nodes that contain a part or the whole name of each one of the drugs.

In [68]:
def extract_drug_nodes(analysis, e1_off, e2_off):
    nodes_drug1 = []
    nodes_drug2 = []
    #Initialize stard and end of each drug
    start_drug1 = e1_off[0][0]
    start_drug2 = e2_off[0][0]
    end_drug1 = e1_off[0][1] if len(e1_off) < 2 else e1_off[len(e1_off) - 1][1]
    end_drug2 = e2_off[0][1] if len(e2_off) < 2 else e2_off[len(e2_off) - 1][1]

    for i_node in range(1, len(analysis.nodes)):
        current_node = analysis.nodes[i_node]
        start = current_node["start_off"]
        end = current_node["end_off"]
        #Append the nodes that are between the start and end
        if start == start_drug1 or (start < start_drug1 <= end):
            nodes_drug1.append(current_node)
        if start == start_drug2 or (start < start_drug2 <= end):
            nodes_drug2.append(current_node)
        if start != start_drug1:
            if end == end_drug1 or (start < end_drug1 <= end) or(start_drug1 < end < end_drug1):
                nodes_drug1.append(current_node)
        if start != start_drug2:
            if end == end_drug2 or (start < end_drug2 <= end) or(start_drug2 < end < end_drug2):
                nodes_drug2.append(current_node)

    return nodes_drug1, nodes_drug2

**Rules without dependency**

This are the rules that we extract based on analyzing which were the most common words that appear between the two drugs for each type of interaction.

In [69]:
def rules_without_dependency(sentence):
    is_ddi = 0
    ddi_type = "null"
    effect_list = ['administer', 'potentiate', 'prevent', 'effect', 'cause']
    #Look for most common words for each DDI type
    if "effect" in sentence:
        is_ddi = 1
        ddi_type = "effect"
    if "should" in sentence:
        is_ddi = 1
        ddi_type = "advise"
    if "increase" in sentence or "decrease" in sentence or "reduce" in sentence:
        is_ddi = 1
        ddi_type = "mechanism"

    return is_ddi, ddi_type

**Check interaction**

This is the function that is in charge of deciding whether there is an interaction between two drugs and if that is the case which type of interaction is. Several rules have been tested using more complex information and others with just looking at the word level, however in this function there are just the ones that achieved a decent F1 score on all datasets(Train, Devel and Test)

In [2]:
def check_interaction(analysis, entities, id_e1, id_e2):
    #Extract the text between the pair of drugs
    inbetween_text = extract_inbetween_text(analysis, entities, id_e1, id_e2)
    
    #Extract if is ddi and the type(less restrictive rule)
    (is_ddi, ddi_type) = rules_without_dependency(inbetween_text)

    e1_off = entities[id_e1]
    e2_off = entities[id_e2]
    
    #Extract more complex information using the dependency tree relations
    nodes_drug1, nodes_drug2 = extract_drug_nodes(analysis, e1_off, e2_off)
    drug1, drug2 = extract_drug_names(entities, id_e1, id_e2, nodes_drug1, nodes_drug2)
    head_drug1, head_drug2 = extract_head_drugs_info(analysis, nodes_drug1, nodes_drug2)
    distance = extract_distance_between_drugs(nodes_drug1, nodes_drug2)
    common_ancestor_info = find_common_ancestor(analysis, nodes_drug1[0]["address"], nodes_drug2[0]["address"])
    
    #If the drugs are the same they can't have an interaction
    if drug1 == drug2:
        return 0, 'null'
    
    # Check the common ancestor and its tag (special case for Int type)
    if common_ancestor_info[0] in ['interact', 'interaction']:
        if common_ancestor_info[1] in ['NN', 'VB', 'NNS', 'VBP']:
            return 1, 'int'
        
    # Check if root contains any of the most common words for advise interaction type
    if analysis.root['lemma'] in ['advise', 'recommend', 'contraindicate', 'suggest']:
        return 1, 'advise'
    
    # Check if root contains any of the most common words for effect interaction type
    if analysis.root['lemma'] in ['enhance', 'inhibit', 'block', 'produce']:
        return 1, 'effect'
    
    # If none of the previous rules is fired, then the basic rules output is used
    if is_ddi:
        return is_ddi, ddi_type

    return 0, 'null'

## Raw output of the Semeval evaluator on DEVEL dataset

## Raw output of the Semeval evaluator on TEST dataset

## Conclusions

As it can be seen in the evaluator output we have achieved the goal of this task which was to obtain a F1 score of 0.15 in Devel dataset. In fact, we have doubled the minimum value achieving a F1 score of 0.32.

With the development of this task we have seen the importance of the dependecy tree to properly extract relations between words and without using this information we are sure that this task would have been much harder. The use of the dependency tree allowed us to extract the connection between tokens of the sentences even with unordered sentences or with passive forms. The most difficult type to detect was the int type due to the distribution of the types, being int the less representative class. 

During the development of this tasks we tested several rules, starting with quite simple ones which just look at the token (word) level without considering relations. However the great improvement came when we started considering the relations and finding which tokens(nodes) had in common.

The metrics obtained in each one of the datasets is quite similar (0.33 on training, 0.32 on Devel and 0,299 on Test ) which implies that the rules extracted are quite generic and cover pretty well the patterns of each interaction type.

During the process of rule selection we tested several values for each rule but our criteria was to left only the generic rules that performed well on all the splits of the data. To extract the knowledge of the most common values we have used automatic functions to describe the distribution of those values for each type and in some cases we have helped with visualizations charts to see at a glance if there were clear patterns (like in the distance between drugs). 

To conclude we would like to emphasize that we achieved decent results with very few rules and even though more rules could be applied we decided just to keep as few as possible while preserving a good performance. However, if further improvements should be made, we could start including more complex patterns like the ones used for Task 4. 
